In [1]:
import os
import sys

import torch
import pandas as pd
import dotenv
import mlflow
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import plotly.graph_objects as go
from huggingface_hub import login

sys.path.append("..")

from utils import calculate_metrics, TrainingConfig

dotenv.load_dotenv("../../.env")

token = os.environ["HF_TOKEN"]
login(token=token)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Time_Series_Forecasting");

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
data_dir = '../../data/panama-electricity-load-forecasting/processed/'

train_df = pd.read_parquet(os.path.join(data_dir, 'train.parquet'))
test_df = pd.read_parquet(os.path.join(data_dir, 'test.parquet'))

test_len = test_df['datetime'].max() - test_df['datetime'].min()
# val_df = train_df[train_df['datetime'] >= train_df['datetime'].max() - test_len]
# train_df = train_df[train_df['datetime'] < train_df['datetime'].max() - test_len]

In [3]:
train_df_chronos = train_df[['datetime', 'nat_demand']].rename(columns={'datetime': 'timestamp', 'nat_demand': 'target'}).copy()
train_df_chronos['item_id'] = 0

# val_df_chronos = val_df[['datetime', 'nat_demand']].rename(columns={'datetime': 'timestamp', 'nat_demand': 'target'}).copy()
# val_df_chronos['item_id'] = 0

test_df_chronos = test_df[['datetime', 'nat_demand']].rename(columns={'datetime': 'timestamp', 'nat_demand': 'target'}).copy()
test_df_chronos['item_id'] = 0

train_data = TimeSeriesDataFrame.from_data_frame(train_df_chronos)
# val_data = TimeSeriesDataFrame.from_data_frame(val_df_chronos)
test_data = TimeSeriesDataFrame.from_data_frame(test_df_chronos)

In [4]:
config = TrainingConfig(
    prediction_length=64,
    eval_metric="MAPE",
    artifact_path="../../models/bolt_base_finetune",
    model_path="autogluon/chronos-bolt-base",
    epochs=10,
    learning_rate=1e-5,
    batch_size=32,
    time_limit=30 * 60,
    context_length=512,
)

predictor = TimeSeriesPredictor(
        prediction_length=config.prediction_length,
        eval_metric=config.eval_metric,
        path=config.artifact_path
        ).fit(
            train_data=train_data,
            # tuning_data=val_data,
            verbosity=4,
            hyperparameters={
                "Chronos": [{
                    "model_path": config.model_path,
                    "fine_tune": True,
                    "fine_tune_lr": config.learning_rate,
                    "fine_tune_batch_size": config.batch_size,
                    "epochs": config.epochs,
                    "fine_tune_steps": config.epochs * (len(train_data) // config.batch_size),
                    "context_length": config.context_length,
                    "num_samples": config.num_samples,
                    "device": "cuda" if torch.cuda.is_available() else 'cpu',
                    "eval_during_fine_tune": True,
                    "ag_args": {"name_suffix": "FineTuned"},
                    "fine_tune_trainer_kwargs": {
                        "evaluation_strategy": "steps",
                        "eval_steps": 50,
                        "save_strategy": "steps",
                        "save_steps": 50,
                        "logging_steps": 50,
                        "gradient_accumulation_steps": 2,
                        "disable_tqdm": False
                    }
                }]
            },
            time_limit=config.time_limit
        )

Beginning AutoGluon training... Time limit = 1800s
AutoGluon will save models to '/home/nikita/projects/time_series_analysis/models/bolt_base_finetune'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan 15 19:18:46 UTC 2
CPU Count:          12
GPU Count:          1
Memory Avail:       21.45 GB / 30.95 GB (69.3%)
Disk Space Avail:   181.31 GB / 233.67 GB (77.6%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'FineTuned'},
                                  'context_length': 512,
                                  'device': 'cuda',
                                  'epochs': 10,
                                  'eval_during_fine_tune': True,
                                  'fine_tune': True,
                                  'fine_t

Step,Training Loss,Validation Loss
50,32.546700,23.890072
100,11.236100,21.046303
150,10.896500,20.738695
200,10.960000,20.098679
250,10.665200,22.405027
300,10.607500,20.794533
350,10.312400,21.079678
400,10.143100,21.557858
450,10.000700,19.968920
500,9.962200,19.772114


{'loss': 32.5467, 'grad_norm': 18.44110107421875, 'learning_rate': 9.963423555230432e-06, 'epoch': 0.0036576444769568397}

***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 32
{'eval_loss': 23.890071868896484, 'eval_runtime': 0.0153, 'eval_samples_per_second': 65.488, 'eval_steps_per_second': 65.488, 'epoch': 0.0036576444769568397}
Saving model checkpoint to /home/nikita/projects/time_series_analysis/models/bolt_base_finetune/models/ChronosFineTuned[autogluon__chronos-bolt-base]/W0/transformers_logs/checkpoint-50
Configuration saved in /home/nikita/projects/time_series_analysis/models/bolt_base_finetune/models/ChronosFineTuned[autogluon__chronos-bolt-base]/W0/transformers_logs/checkpoint-50/config.json
Model weights saved in /home/nikita/projects/time_series_analysis/models/bolt_base_finetune/models/ChronosFineTuned[autogluon__chronos-bolt-base]/W0/transformers_logs/checkpoint-50/model.safetensors
{'loss': 11.2361, 'grad_norm': 17.60593032836914, 'learning_rate': 9.

KeyboardInterrupt: 

In [6]:
predictor = TimeSeriesPredictor(
        path=config.artifact_path
        ).fit(
            train_data=train_data,
        )

Beginning AutoGluon training...
AutoGluon will save models to '/home/nikita/projects/time_series_analysis/models/bolt_base_finetune'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan 15 19:18:46 UTC 2
CPU Count:          12
GPU Count:          1
Memory Avail:       20.82 GB / 30.95 GB (67.3%)
Disk Space Avail:   180.54 GB / 233.67 GB (77.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'h'
Provided train_data has 43775 rows, 1 time series. Medi

In [7]:
predictions = predictor.predict(train_data)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [10]:
predictions

,,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
item_id,timestamp,,,,,,,,,,
0,2020-01-01,1049.70352,1036.497704,1042.946071,1045.441614,1046.905426,1049.70352,1052.690848,1053.032192,1057.50766,1064.28688


In [8]:
train_size = 500
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=train_df["datetime"][-train_size:],
        y=train_df["nat_demand"][-train_size:],
        name="Train",
        line=dict(color="#6495ED"),
    )
)
fig.add_trace(
    go.Scatter(
        x=test_df["datetime"],
        y=test_df["nat_demand"],
        name="Test",
        line=dict(color="#50C878"),
    )
)
fig.add_trace(
    go.Scatter(
        x=test_df["datetime"],
        y=predictions["mean"],
        name="Chronos Forecast (mean)",
        line=dict(color="#D70040", dash="dot"),
    )
)

fig.add_trace(go.Scatter(
    x=test_df['datetime'],
    y=predictions['0.9'],
    mode='lines',
    line=dict(width=0),
    name='90th percentile',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=test_df['datetime'],
    y=predictions['0.1'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 127, 14, 0.6)',
    line=dict(width=0),
    name='Chronos Confidence Interval (0.1-0.9)'
))

fig.update_layout(
    title="Target value",
    xaxis_title="Date",
    yaxis_title="National Demand",
    template="plotly_white",
    width=1_400,
    height=500,
)
fig.show()

In [ ]:
y_true = test_df['nat_demand']
y_pred = predictions['mean']

metrics = calculate_metrics(y_true=y_true, y_pred=y_pred)

exp_name = 'Time_Series_Forecasting'
run_name = 'Chronos_FineTuned_SingleSeries'
mlflow.set_experiment(exp_name)
with mlflow.start_run(run_name=run_name):
    mlflow.log_params(config.__dict__)
    mlflow.log_metrics(metrics)

2025/03/31 00:19:15 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



🏃 View run Chronos_FineTuned_SingleSeries at: http://127.0.0.1:5000/#/experiments/185045746886025740/runs/f21a3ae1bc9a4f96851c5467d48c2693
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/185045746886025740
